## Experimental design - the basics

Previusly to write any code or do statistics, we need to planning step by step our experimental design:
* Business goal, first we determine the business goal for the experiment, generally this involve a variable like, cost, costumer retention, etc.
* Target metric, we try to pick a metric as close to the intervention, capable to reduce extraneous noise.
* Define the intervention (test the smallest posible).
* Behavioral logic: Articulate a reasonable behavioral story.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns
import statsmodels.stats.proportion as ssprop
import statsmodels.stats.power as ssp

In [2]:
hist_data_df = pd.read_csv('data/historical_data.csv')
exp_data_df = pd.read_csv('data/experimental_data.csv')

In [3]:
hist_data_df.head()

,age,gender,period,seasonality,month,booked
0,34,male,1,0.749993,2,0
1,39,male,1,0.749993,2,0
2,42,male,1,0.749993,2,0
3,33,male,1,0.749993,2,0
4,32,male,1,0.749993,2,1


In [4]:
exp_data_df.head()

,age,gender,period,seasonality,month,booked,oneclick
0,47,male,33,6.492170e-08,10,0,1
1,49,male,33,6.492170e-08,10,0,1
2,37,male,33,6.492170e-08,10,0,0
3,44,male,33,6.492170e-08,10,0,1
4,45,male,33,6.492170e-08,10,0,1


#### Determening random assignment

In [5]:
K = 2 # (groups:control and treatment)
assgnt = np.random.uniform(0,1,1) # (we use uniform distribution to assign a group)
group = 'control' if assgnt <= 1/K else 'treatment'

B.E.A.N (B is for beta {statistical significance}, E for effect size, A for alpha{statistical power is often represented as 1 − α}, and N for sample size) referred to the four variables that determining the "statistical power analysis"

In [6]:
# Set a the sample size
effect_size = ssprop.proportion_effectsize(0.194, 0.184) #we try to increase reservation rate at least 1%
ssp.tt_ind_solve_power(effect_size = effect_size, 
                      alpha = 0.05, 
                      nobs1 = None,
                      alternative = 'larger',
                      power=0.8) #Conventional statistical power

18950.818821249803

In [7]:
#running a logistic regression with a small sample
exp_null_data_df = hist_data_df.copy().sample(2000)
exp_null_data_df['oneclick'] = np.where(np.random.uniform(0,1,2000)>0.5,1,0)
mod = smf.logit('booked ~ oneclick + age + gender', data = exp_null_data_df)
mod.fit(disp=0).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                 2000
Model:                          Logit   Df Residuals:                     1996
Method:                           MLE   Df Model:                            3
Date:                Wed, 10 Nov 2021   Pseudo R-squ.:                  0.2625
Time:                        17:58:25   Log-Likelihood:                -698.63
converged:                       True   LL-Null:                       -947.32
Covariance Type:            nonrobust   LLR p-value:                1.762e-107
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          9.9435      0.642     15.478      0.000       8.684      11.203
gender[T.male]     0.2524      0.137      1.849      0.065      -0.015       0.520
oneclick           0.0872      0.135      0.644      0.520      -0.178       0.353
age               -0.3121      0.018    -17.588      0.000      -0.347      -0.277
==================================================================================
"""

The traditional rule would be to consider the impact of the 1-click button to be significant and implement it if the corresponding coefficient had a p-value less than 0.1. Because it's aprox 0.225, we would consider the effect not significant and decline to implement the button.

#### Bootstrap simulations

Bootstrap simulations offer an alternative that better reflects the realities and needs of applied data analysis. When using bootstrap simulations, our decision rule doesn't rely on p-values... Simulations offer a very versatile but transparent way of determining the necessary
sample size for any experiment, however weird the data or complex the business decision at hand.

In [8]:
def log_reg_fun(dat_df):
    
    model = smf.logit('booked ~ oneclick + age + gender', data = dat_df)
    res = model.fit(disp=0)
    coeff = res.params['oneclick']
    return coeff

def boot_CI_fun(dat_df, metric_fun, B = 100, conf_level = 0.9):
    N= len(dat_df)
    conf_level = conf_level
    coeffs = []
    
    for i in range(B):
        sim_data_df = dat_df.sample(n=N, replace = True)
        coeff = metric_fun(sim_data_df)
        coeffs.append(coeff)
        
    coeffs.sort()
    start_idx = round(B* (conf_level) / 2)
    end_idx =- round(B*(1 - conf_level)/2)
    confint = [coeffs[start_idx], coeffs[end_idx]]
    return(confint)

def decision_fun(dat_df, metric_fun, B = 100, conf_level = 0.9):
    boot_CI = boot_CI_fun(dat_df, metric_fun, B=B, conf_level = conf_level)
    decision = 1 if boot_CI[0]>0 else 0
    return decision

def single_sim_fun(Nexp, dat_df = hist_data_df, metric_fun = log_reg_fun, 
                   eff_size = 0.01, B = 100, conf_level = 0.9):
    
    #Adding predicted probability of booking
    hist_model = smf.logit('booked ~ age + gender + period', data = dat_df)
    res = hist_model.fit(disp=0)
    sim_data_df = dat_df.copy()
    sim_data_df['pred_prob_bkg'] = res.predict()
    #Filtering down to desired sample size
    sim_data_df = sim_data_df.sample(Nexp)
    #Random assignment of experimental groups
    sim_data_df['oneclick'] = np.where(np.random.uniform(size=Nexp) <= 0.5, 0, 1)
    # Adding effect to treatment group
    sim_data_df['pred_prob_bkg'] = np.where(sim_data_df.oneclick == 1, 
                                            sim_data_df.pred_prob_bkg + eff_size, 
                                            sim_data_df.pred_prob_bkg)
    sim_data_df['booked'] = np.where(sim_data_df.pred_prob_bkg >= \
                                     np.random.uniform(size=Nexp), 1, 0)
    
    #Calculate the decision (we want it to be 1)
    decision = decision_fun(sim_data_df, metric_fun = metric_fun, B = B, 
                            conf_level = conf_level)
     
    return decision

def power_sim_fun(dat_df, metric_fun, Nexp, eff_size, Nsim, B = 100,
                 conf_level = 0.9):
    power_lst = []
    for i in range (Nsim):
        power_lst.append(single_sim_fun(Nexp=Nexp, dat_df = dat_df,
                                       metric_fun = metric_fun,
                                       eff_size = eff_size, B =B,
                                       conf_level = conf_level))
        
        power = np.mean(power_lst)
        return(power)


In [9]:
single_sim_fun(Nexp = 1000)

1

In [10]:
power_sim_fun(dat_df=hist_data_df, metric_fun = log_reg_fun, Nexp = int(4e4), 
              eff_size=0.01, Nsim=20)

1.0

In [11]:
#Alternative parallelized function for higher speed
from joblib import Parallel, delayed
import psutil

def opt_power_sim_fun(dat_df, metric_fun, Nexp, eff_size, Nsim, B = 100, conf_level = 0.9):
    #Parallelized version with joblib
    n_cpu = psutil.cpu_count() #Counting number of cores on machine
    counter = [Nexp] * Nsim
    res_parallel = Parallel(n_jobs = n_cpu)(delayed(single_sim_fun)(Nexp) for Nexp in counter)
    pwr = np.mean(res_parallel)
    return(pwr)
opt_power_sim_fun(dat_df=hist_data_df, metric_fun = log_reg_fun, Nexp = int(1e3), eff_size=0.01, Nsim=10)

0.8

### Analyzing and interpreting experimental results

In [12]:
log_mod_exp = smf.logit('booked ~ age + gender + oneclick', data = exp_data_df)
res = log_mod_exp.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.161220
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 booked   No. Observations:                40160
Model:                          Logit   Df Residuals:                    40156
Method:                           MLE   Df Model:                            3
Date:                Wed, 10 Nov 2021   Pseudo R-squ.:                  0.3311
Time:                        17:58:58   Log-Likelihood:                -6474.6
converged:                       True   LL-Null:                       -9679.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         11.6928      0.226     51.819      0.000      11.251      12.135
gender[T.male]     0.2542      0.049      5.182      0.000       0.158       0.350
age               -0.3941      0.006    -61.282      0.000      -0.407      -0.381
oneclick           0.1578      0.047      3.357      0.001       0.066       0.250
==================================================================================
"""

In [13]:
### Calculating average difference in probabilities
def diff_prob_fun(dat_df, reg_model = log_mod_exp):
    
    #Creating new copies of data
    no_button_df = dat_df.loc[:, 'age':'gender']
    no_button_df.loc[:, 'oneclick'] = 0
    button_df = dat_df.loc[:,'age':'gender']
    button_df.loc[:, 'oneclick'] = 1
    
    #Adding the predictions of the model 
    no_button_df.loc[:, 'pred_bkg_rate'] = res.predict(no_button_df)
    button_df.loc[:, 'pred_bkg_rate'] = res.predict(button_df)
    
    diff = button_df.loc[:,'pred_bkg_rate'] \
    - no_button_df.loc[:,'pred_bkg_rate']
    return diff.mean()
    
diff_prob_fun(exp_data_df, reg_model = log_mod_exp)

0.007129714313552685

With the last result we can see an positive effect (0.0071%) of the implementation of the button, this is lower than our target set in 1%. 

In [14]:
boot_CI_fun(exp_data_df, diff_prob_fun, B = 100, conf_level = 0.9)

[0.0071298982086282535, 0.007209905800872517]

The bootstrap interval is narrow and doesn't cross the zero barrier. Therefore, we can treat our result as empirically statistically significant at the 5% level... also we can treat our result as significant. So, implement the button or not it'll depend on estimated effect size, cost, and risk appetite.